<header> Mimic III v1.4 Investigation </header> 

### Part 1: Import the data

All data has been downloaded from the Mimic-III website:

MIMIC-III, a freely accessible critical care database. Johnson AEW, Pollard TJ, Shen L, Lehman L, Feng M, Ghassemi M, Moody B, Szolovits P, Celi LA, and Mark RG. Scientific Data (2016). DOI: 10.1038/sdata.2016.35. Available from: http://www.nature.com/articles/sdata201635

Access can be granted via PhysioNetWorks:

Goldberger AL, Amaral LAN, Glass L, Hausdorff JM, Ivanov PCh, Mark RG, Mietus JE, Moody GB, Peng C-K, Stanley HE. PhysioBank, PhysioToolkit, and PhysioNet: Components of a New Research Resource for Complex Physiologic Signals. Circulation 101(23):e215-e220 [Circulation Electronic Pages; http://circ.ahajournals.org/content/101/23/e215.full]; 2000 (June 13).

First we need to import the needed modules.  Let us test psycopg2 which will allow us to access our PSQL database using Python.

In [1]:
#Import necessary modules

import psycopg2
import psycopg2.extras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import roc_auc_score, accuracy_score


#Connect to PSQL database
try:
    conn = psycopg2.connect(dbname="mimic", user="postgres", password="Postgres89")
except:
    print "I am unable to connect to the database"

#Create cursor to access tables from database    
curr = conn.cursor()
try:
    curr.execute("SELECT LOS from ICUSTAYS")
except:
    print "I cannot SELECT from ADMISSIONS"

#Fetch all rows from table    
rows = curr.fetchall()
print rows[1]
rowcount = curr.rowcount
print "Length is ", rowcount
print "\nRows: \n"

#Example creating groups of patients based on length of stay (LOS)
none_count =0
los_sum = 0.
los_groups = {"<1 day": 0, "1-2 days": 0, "2-3 days": 0, "3-5 days": 0, "5-10 days":0, "10+ days": 0}
for record in rows:
    if record[0] is not None:
        los_sum = los_sum + record[0]
        if record[0] <= 1.0:
            los_groups["<1 day"] = los_groups["<1 day"]+1
        elif record[0]<=2.0:
            los_groups["1-2 days"] = los_groups["1-2 days"]+1
        elif record[0]<=3.0:
            los_groups["2-3 days"] = los_groups["2-3 days"]+1
        elif record[0]<=5.0:
            los_groups["3-5 days"] = los_groups["3-5 days"]+1
        elif record[0]<=10.0:
            los_groups["5-10 days"] = los_groups["5-10 days"]+1
        else:
            los_groups["10+ days"] = los_groups["10+ days"]+1
        
    else:
        none_count += 1
        
        
los_average = los_sum/(rowcount-none_count)
print "Number of rows with no LOS data", none_count
print "Average Length of Stay is ", los_average
print los_groups

#Close cursor after finished accessing table
curr.close()

(3.2788,)
Length is  61532

Rows: 

Number of rows with no LOS data 10
Average Length of Stay is  4.9179715809
{'10+ days': 6771, '2-3 days': 9644, '<1 day': 12311, '3-5 days': 8801, '1-2 days': 16901, '5-10 days': 7094}


In [2]:
#Access relevant fields from ADMISSIONS table
curr2 = conn.cursor()
try:
    curr2.execute("SELECT SUBJECT_ID, HADM_ID, ADMITTIME, DISCHTIME, DEATHTIME, INSURANCE, LANGUAGE, RELIGION, MARITAL_STATUS, ETHNICITY from ADMISSIONS")
except:
    print "I cannot SELECT from ADMISSIONS"

#Get column names for Pandas DataFrame
admcolnames = [desc[0] for desc in curr2.description]    
#Create Pandas DataFrame
dfadm = pd.DataFrame(curr2.fetchall(), columns=admcolnames)
#print admcolnames
print dfadm.head()
print len(dfadm.index)
curr2.close()

   subject_id  hadm_id           admittime           dischtime deathtime  \
0          22   165315 2196-04-09 12:26:00 2196-04-10 15:54:00      None   
1          23   152223 2153-09-03 07:15:00 2153-09-08 19:10:00      None   
2          23   124321 2157-10-18 19:34:00 2157-10-25 14:00:00      None   
3          24   161859 2139-06-06 16:14:00 2139-06-09 12:48:00      None   
4          25   129635 2160-11-02 02:06:00 2160-11-05 14:55:00      None   

  insurance language           religion marital_status ethnicity  
0   Private     None       UNOBTAINABLE        MARRIED     WHITE  
1  Medicare     None           CATHOLIC        MARRIED     WHITE  
2  Medicare     ENGL           CATHOLIC        MARRIED     WHITE  
3   Private     None  PROTESTANT QUAKER         SINGLE     WHITE  
4   Private     None       UNOBTAINABLE        MARRIED     WHITE  
58976


In [3]:
#Access relevant fields from PATIENTS table
curr3 = conn.cursor()
try:
    curr3.execute("SELECT SUBJECT_ID, GENDER, DOB, DOD, EXPIRE_FLAG from PATIENTS")
except:
    print "I cannot SELECT from PATIENTS"

#Create PATIENTS DataFrame
patcolnames = [desc[0] for desc in curr3.description]
dfpat = pd.DataFrame(curr3.fetchall(), columns=patcolnames)
print dfpat.head()
print len(dfpat.index)
curr3.close()

   subject_id gender        dob        dod  expire_flag
0         249      F 2075-03-13        NaT            0
1         250      F 2164-12-27 2188-11-22            1
2         251      M 2090-03-15        NaT            0
3         252      M 2078-03-06        NaT            0
4         253      F 2089-11-26        NaT            0
46520


In [4]:
#Merge ADMISSIONS and PATIENTS table
dfadmpat = dfpat.merge(dfadm, how='inner', on="subject_id")
print dfadmpat.head(n=5)
print len(dfadmpat.index)

   subject_id gender        dob        dod  expire_flag  hadm_id  \
0         249      F 2075-03-13        NaT            0   116935   
1         249      F 2075-03-13        NaT            0   149546   
2         249      F 2075-03-13        NaT            0   158975   
3         250      F 2164-12-27 2188-11-22            1   124271   
4         251      M 2090-03-15        NaT            0   117937   

            admittime           dischtime            deathtime insurance  \
0 2149-12-17 20:41:00 2149-12-31 14:55:00                 None  Medicare   
1 2155-02-03 20:16:00 2155-02-14 11:15:00                 None  Medicare   
2 2156-04-27 15:33:00 2156-05-14 15:30:00                 None  Medicare   
3 2188-11-12 09:22:00 2188-11-22 12:00:00  2188-11-22 12:00:00  Self Pay   
4 2110-07-27 06:46:00 2110-07-29 15:23:00                 None   Private   

  language       religion marital_status               ethnicity  
0     None       CATHOLIC       DIVORCED                   WHITE  


In [5]:
#Access relevant fields from ICUSTAYS table
curr4 = conn.cursor()
try:
    curr4.execute("SELECT SUBJECT_ID, HADM_ID, LOS from ICUSTAYS")
except:
    print "I cannot SELECT from ICUSTAYS"

#Create ICUSTAYS DataFrame    
icucolnames = [desc[0] for desc in curr4.description]
dficu = pd.DataFrame(curr4.fetchall(), columns=icucolnames)
print len(dficu.index)
print dficu.head()

#Merge 
dfmimic = dfadmpat.merge(dficu, how="inner", on=['hadm_id', 'subject_id'])
print len(dfmimic.index)
#Drop duplicate data, it can be assumed if there are multiple admissions
#of the same patient, they will have survived the earlier admissions.
dfmimic.drop_duplicates('hadm_id', inplace=True) #Drop duplicate hospital admission data
print len(dfmimic.index)
dfmimic.drop_duplicates('subject_id', keep='last', inplace=True) #Drop duplicate subject data
print len(dfmimic.index)



curr4.close()

61532
   subject_id  hadm_id     los
0         268   110404  3.2490
1         269   106296  3.2788
2         270   188028  2.8939
3         271   173727  2.0600
4         272   164716  1.6202
61532
57786
46476


Let's get counts by gender ('M' or 'F') and whether they lived or died.

In [6]:
print len(dfmimic.index)

#Group by gender
mimic_by_gender = dfmimic.groupby('gender') #DataFrame grouped by gender
print mimic_by_gender.head()
print mimic_by_gender['expire_flag'].sum() #Sum of deaths
print mimic_by_gender['expire_flag'].value_counts() #Counts of deaths vs no deaths by gender
print dfmimic['expire_flag'].sum()
print dfmimic['expire_flag'].value_counts(dropna = False)

46476
    subject_id gender        dob        dod  expire_flag  hadm_id  \
3          249      F 2075-03-13        NaT            0   158975   
5          250      F 2164-12-27 2188-11-22            1   124271   
6          251      M 2090-03-15        NaT            0   117937   
8          252      M 2078-03-06        NaT            0   193470   
9          253      F 2089-11-26        NaT            0   176189   
10         255      M 2109-08-05        NaT            0   112013   
14         256      M 2086-07-31        NaT            0   108811   
16         257      F 2031-04-03 2121-07-08            1   179006   
17         258      F 2124-09-19        NaT            0   189406   
20         261      M 2025-08-04 2102-06-29            1   118523   

             admittime           dischtime            deathtime insurance  \
3  2156-04-27 15:33:00 2156-05-14 15:30:00                 None  Medicare   
5  2188-11-12 09:22:00 2188-11-22 12:00:00  2188-11-22 12:00:00  Self Pay   
6  

We can see that our table contains data for 30739 patients that survived their stay in the ICU and 15737 patients who died in the ICU.
Let's create a benchmark model for the data that will guess all patients survived.  We will use scikit-learn's AUC function to compute the area under the curve.  

In [7]:
#Create NumPy array predicted all '1's for patient survival
mimic_length = len(dfmimic.index)
y_scores = np.ones(mimic_length, dtype=np.int)
y_true = dfmimic['expire_flag']
print roc_auc_score(y_true, y_scores)
print roc_auc_score(y_true, y_true)

 


0.5
1.0


Now we can find the patient's age on admission by subtracting the patient's date of birth ('dob') from the hospital's admission date ('admittime').  Patient's will be grouped by aging similarly to how the CDC grouped patient's in the National Hospital Discharge Survey 2000-2010: https://www.cdc.gov/nchs/data/databriefs/db118.htm#x2013;2010%3C/a%3E%20

In [8]:
from datetime import timedelta

def groupage(admittime, dob):
    '''
    inputs:
        -admittime: A datetime value of when the patient was admitted to the ICU
        -dob: A datetime value of the patient's Date of Birth
    output:
        The group a patient belongs to based on the difference between admittime and dob
    '''
    age_in_years = (admittime-dob)
    #Group ages based on CDC grouping
    if age_in_years >= timedelta(85*365.242):
        return '85 years and over'
    elif age_in_years >= timedelta(75*365.242):
        return '75-84 years'
    elif age_in_years >= timedelta(65*365.242):
        return '65-74 years'
    elif age_in_years >= timedelta(45*365.242):
        return '45-64 years'
    else:
        return 'Under 45 years'
    

dfmimic['age'] = dfmimic.apply(lambda row: groupage(row['admittime'], row['dob']), axis=1)
print dfmimic['age'].value_counts()
print dfmimic.head()    
    
    

Under 45 years       13620
45-64 years          12833
75-84 years           8080
65-74 years           7678
85 years and over     4265
Name: age, dtype: int64
   subject_id gender        dob        dod  expire_flag  hadm_id  \
3         249      F 2075-03-13        NaT            0   158975   
5         250      F 2164-12-27 2188-11-22            1   124271   
6         251      M 2090-03-15        NaT            0   117937   
8         252      M 2078-03-06        NaT            0   193470   
9         253      F 2089-11-26        NaT            0   176189   

            admittime           dischtime            deathtime insurance  \
3 2156-04-27 15:33:00 2156-05-14 15:30:00                 None  Medicare   
5 2188-11-12 09:22:00 2188-11-22 12:00:00  2188-11-22 12:00:00  Self Pay   
6 2110-07-27 06:46:00 2110-07-29 15:23:00                 None   Private   
8 2133-08-15 04:23:00 2133-08-19 17:30:00                 None   Private   
9 2174-01-21 20:58:00 2174-01-26 16:15:00           

Just as patient's were grouped into different age groups, patients will now be grouped by length of ICU stay based the CDC's National Hospital Discharge Survey 2000-2010: https://www.cdc.gov/nchs/data/databriefs/db118.htm#x2013;2010%3C/a%3E%20

In [9]:
def grouplos(los):
    '''
    inputs:
        -los: patient length of stay in ICU
    outputs: 
        Group based on patient length of stay
    '''
    if los >= 10:
        return '10 or more days'
    elif los >= 8:
        return '8-9 days'
    elif los >= 6:
        return '6-7 days'
    elif los >= 4:
        return '4-5 days'
    elif los >= 2:
        return '2-3 days'
    else:
        return '1 day'

dfmimic['los'] = dfmimic.apply(lambda row: grouplos(row['los']), axis=1)
print dfmimic['los'].value_counts()
    

1 day              23078
2-3 days           10733
10 or more days     5171
4-5 days            4039
6-7 days            2127
8-9 days            1328
Name: los, dtype: int64


Let's visualize the data using matplotlib

In [10]:

'''
N=2
diemw = (mendie, womdie)
livemw = (menlive, womlive)

ind = np.arange(N) #x locations for bars
width = 0.35 #width of bars

fig, ax = plt.subplots()
rects1 = ax.bar(ind, diemw, width, color='r')
rects2 = ax.bar(ind+width, livemw, width, color='g')

#Label axes and graph
ax.set_ylabel('Number of Patients')
ax.set_title('Patient deaths by Gender')
ax.set_xticks(ind + width / 2)
ax.set_xticklabels(('Men', 'Women'))

ax.legend((rects1[0], rects2[0]), ('Deaths', 'Lives'))

def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                '%d' % int(height),
                ha='center', va='bottom')
        
autolabel(rects1)
autolabel(rects2)

plt.show()
'''

'\nN=2\ndiemw = (mendie, womdie)\nlivemw = (menlive, womlive)\n\nind = np.arange(N) #x locations for bars\nwidth = 0.35 #width of bars\n\nfig, ax = plt.subplots()\nrects1 = ax.bar(ind, diemw, width, color=\'r\')\nrects2 = ax.bar(ind+width, livemw, width, color=\'g\')\n\n#Label axes and graph\nax.set_ylabel(\'Number of Patients\')\nax.set_title(\'Patient deaths by Gender\')\nax.set_xticks(ind + width / 2)\nax.set_xticklabels((\'Men\', \'Women\'))\n\nax.legend((rects1[0], rects2[0]), (\'Deaths\', \'Lives\'))\n\ndef autolabel(rects):\n    """\n    Attach a text label above each bar displaying its height\n    """\n    for rect in rects:\n        height = rect.get_height()\n        ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,\n                \'%d\' % int(height),\n                ha=\'center\', va=\'bottom\')\n        \nautolabel(rects1)\nautolabel(rects2)\n\nplt.show()\n'

Now let us create a dataframe with the feature we will want to train our model on

In [11]:
dffeatures = dfmimic[['gender', 'ethnicity', 'marital_status', 'insurance', 'language', 'religion', 'expire_flag', 'los', 'age']]
expire = dfmimic['expire_flag']
print dffeatures.head()

  gender               ethnicity marital_status insurance language  \
3      F                   WHITE       DIVORCED  Medicare     ENGL   
5      F  BLACK/AFRICAN AMERICAN         SINGLE  Self Pay     HAIT   
6      M   UNKNOWN/NOT SPECIFIED           None   Private     None   
8      M                   WHITE         SINGLE   Private     None   
9      F                   WHITE        WIDOWED  Medicare     None   

        religion  expire_flag              los             age  
3       CATHOLIC            0         2-3 days     75-84 years  
5  NOT SPECIFIED            1  10 or more days  Under 45 years  
6          OTHER            0            1 day  Under 45 years  
8   UNOBTAINABLE            0         2-3 days     45-64 years  
9       CATHOLIC            0            1 day     75-84 years  


In [12]:
#CDC reports average LOS for patients who died was 7.9 days
#Control for LOS greater than inpatient death average
#print dffeatures['expire_flag'].value_counts(dropna=False)
#dffeatures = dffeatures[dffeatures.los >= 10]
#print dffeatures['expire_flag'].value_counts(dropna=False)

In [13]:
for column in dffeatures:
    print dffeatures[column].value_counts(dropna=False)



M    26096
F    20380
Name: gender, dtype: int64
WHITE                                                       32080
UNKNOWN/NOT SPECIFIED                                        4132
BLACK/AFRICAN AMERICAN                                       3585
HISPANIC OR LATINO                                           1336
ASIAN                                                        1297
OTHER                                                        1288
UNABLE TO OBTAIN                                              787
PATIENT DECLINED TO ANSWER                                    488
ASIAN - CHINESE                                               230
BLACK/CAPE VERDEAN                                            167
HISPANIC/LATINO - PUERTO RICAN                                153
WHITE - RUSSIAN                                               118
MULTI RACE ETHNICITY                                          109
BLACK/HAITIAN                                                  75
WHITE - OTHER EUROPEAN     

Since we will only be looking at rows containing known values, we will clean up our dataframe by removing any rows missing data.

In [14]:
#Drop rows containing NaN values
dffeatures_clean = dffeatures.dropna()
for column in dffeatures_clean:
    print dffeatures_clean[column].value_counts(dropna=False)



M    13463
F    10521
Name: gender, dtype: int64
WHITE                                                       17643
BLACK/AFRICAN AMERICAN                                       1977
HISPANIC OR LATINO                                            799
OTHER                                                         584
UNKNOWN/NOT SPECIFIED                                         516
UNABLE TO OBTAIN                                              469
ASIAN                                                         459
PATIENT DECLINED TO ANSWER                                    243
ASIAN - CHINESE                                               195
BLACK/CAPE VERDEAN                                            161
HISPANIC/LATINO - PUERTO RICAN                                142
WHITE - RUSSIAN                                               115
MULTI RACE ETHNICITY                                           75
BLACK/HAITIAN                                                  70
HISPANIC/LATINO - DOMINICAN

In [15]:
#Drop rows with unknown marital status    
df_clean_marital = dffeatures_clean[dffeatures_clean.marital_status != 'UNKNOWN (DEFAULT)']

#Drop rows with not obtained ethnicity
df_clean_marital_ethnicity = df_clean_marital[df_clean_marital.ethnicity != 'UNABLE TO OBTAIN']
df_clean_marital_ethnicity = df_clean_marital_ethnicity[df_clean_marital_ethnicity.ethnicity != 'PATIENT DECLINED TO ANSWER']
df_clean_marital_ethnicity = df_clean_marital_ethnicity[df_clean_marital_ethnicity.ethnicity != 'UNKNOWN/NOT SPECIFIED']
df_clean_marital_ethnicity = df_clean_marital_ethnicity[df_clean_marital_ethnicity.religion != 'UNOBTAINABLE']
#print df_clean_marital['marital_status'].value_counts(dropna=False)
for column in df_clean_marital_ethnicity:
    print df_clean_marital_ethnicity[column].value_counts(dropna=False)


M    11897
F     9448
Name: gender, dtype: int64
WHITE                                                       16655
BLACK/AFRICAN AMERICAN                                       1857
HISPANIC OR LATINO                                            729
OTHER                                                         521
ASIAN                                                         376
ASIAN - CHINESE                                               173
BLACK/CAPE VERDEAN                                            154
HISPANIC/LATINO - PUERTO RICAN                                136
WHITE - RUSSIAN                                               109
MULTI RACE ETHNICITY                                           71
BLACK/HAITIAN                                                  64
WHITE - OTHER EUROPEAN                                         59
HISPANIC/LATINO - DOMINICAN                                    58
ASIAN - ASIAN INDIAN                                           48
WHITE - BRAZILIAN          

In [16]:
#Ensure there is  a high enough population (>10) for language and religion groups

language_counts = df_clean_marital_ethnicity['language'].value_counts()
religion_counts = df_clean_marital_ethnicity['religion'].value_counts()
#List of groups with <10 members
df_clean_language = df_clean_marital_ethnicity[df_clean_marital_ethnicity['language'].isin(language_counts[language_counts >= 10].index)]
df_clean_groups = df_clean_language[df_clean_language['religion'].isin(religion_counts[religion_counts >= 10].index)]
print df_clean_groups['language'].value_counts()
print df_clean_groups['religion'].value_counts()

ENGL    18882
SPAN      606
RUSS      450
PTUN      282
CANT      215
PORT      172
CAPE      164
MAND       87
HAIT       81
ITAL       71
VIET       55
GREE       45
PERS       21
POLI       20
ARAB       19
AMER       17
CAMB       16
HIND       14
KORE       14
ALBA       10
Name: language, dtype: int64
CATHOLIC                  8473
NOT SPECIFIED             5908
PROTESTANT QUAKER         2828
JEWISH                    2086
OTHER                      856
EPISCOPALIAN               323
CHRISTIAN SCIENTIST        185
GREEK ORTHODOX             183
BUDDHIST                   108
MUSLIM                      64
JEHOVAH'S WITNESS           60
UNITARIAN-UNIVERSALIST      58
HINDU                       39
ROMANIAN EAST. ORTH         38
7TH DAY ADVENTIST           32
Name: religion, dtype: int64


In [17]:
for column in df_clean_groups:
    print df_clean_groups[column].value_counts(dropna=False)
print len(df_clean_groups.columns)

M    11847
F     9394
Name: gender, dtype: int64
WHITE                                                       16628
BLACK/AFRICAN AMERICAN                                       1836
HISPANIC OR LATINO                                            729
OTHER                                                         518
ASIAN                                                         352
ASIAN - CHINESE                                               166
BLACK/CAPE VERDEAN                                            154
HISPANIC/LATINO - PUERTO RICAN                                136
WHITE - RUSSIAN                                               109
MULTI RACE ETHNICITY                                           69
BLACK/HAITIAN                                                  63
WHITE - OTHER EUROPEAN                                         59
HISPANIC/LATINO - DOMINICAN                                    58
ASIAN - ASIAN INDIAN                                           42
WHITE - BRAZILIAN          

In [18]:
curr5 = conn.cursor()
try:
    curr5.execute("SELECT * from DIAGNOSES_ICD")
except:
    print "I cannot SELECT from DIAGNOSES_ICD"
    
diagcolnames = [desc[0] for desc in curr5.description]
df = pd.DataFrame(curr5.fetchall(), columns=diagcolnames)
print df.head()
print df['icd9_code'].value_counts()
print len(df.index)

print df['icd9_code'].head()


   row_id  subject_id  hadm_id  seq_num icd9_code
0    1297         109   172335      1.0     40301
1    1298         109   172335      2.0       486
2    1299         109   172335      3.0     58281
3    1300         109   172335      4.0      5855
4    1301         109   172335      5.0      4254
4019     20703
4280     13111
42731    12891
41401    12429
5849      9119
25000     9058
2724      8690
51881     7497
5990      6555
53081     6326
2720      5930
V053      5779
V290      5519
2859      5406
2449      4917
486       4839
2851      4552
2762      4528
496       4431
99592     3912
V5861     3806
0389      3725
5070      3680
V3000     3566
5859      3435
311       3431
40390     3421
3051      3358
412       3278
2875      3065
         ...  
9595         1
67401        1
61800        1
75435        1
61804        1
29625        1
8281         1
0338         1
20011        1
20018        1
V1869        1
66131        1
83503        1
28244        1
44620        1
28243     

In [19]:
df_rel_lan = df_clean_groups[['religion', 'language']]
print df_rel_lan.head()

         religion language
3        CATHOLIC     ENGL
5   NOT SPECIFIED     HAIT
10  NOT SPECIFIED     ENGL
14  NOT SPECIFIED     ENGL
24          OTHER     ENGL


### Part 2: Building Benchmark Model.
First we will start by building a benchmark model using only patient age and length of stay.  We will use only these two features as higher patient age and length of stay are consistently shown to have higher mortality rates.

In [20]:
#Select raw features
benchmark_features_raw = dfmimic[['age', 'los']]
#Select target (patient death)
expiry = dfmimic['expire_flag']

#Use one hot encoding to convert age and LOS groups to dummy variables
benchmark_features = pd.get_dummies(benchmark_features_raw)
print list(benchmark_features.columns)



['age_45-64 years', 'age_65-74 years', 'age_75-84 years', 'age_85 years and over', 'age_Under 45 years', 'los_1 day', 'los_10 or more days', 'los_2-3 days', 'los_4-5 days', 'los_6-7 days', 'los_8-9 days']


Use sklearn to split the data into test and train data.

In [21]:
from sklearn.model_selection import train_test_split

bench_train, bench_test, exp_train, exp_test = train_test_split(benchmark_features, expiry, random_state=42)

#Check for correct splitting
print "Training set has {} samples.".format(bench_train.shape[0])
print "Testing set has {} samples.".format(bench_test.shape[0])

Training set has 34857 samples.
Testing set has 11619 samples.


Create a training and predicting function to test different machine learning algorithms

In [22]:
def train_results(learner, sample_size, X_train, y_train, X_test, y_test):
    '''
    inputs:
        -learner: type of learning algorithm to be compared
        -sample_size: percent of data being trained
        -X_train: features training set
        -y_train: expiry training set
        -X_test: features test set
        -y_test: expiry training set
    '''
    
    results = {}
    
    #Fit the learner to the training data using slicing with 'sample_size'
    start = time() # Get start time
    learner = learner.fit(X_train[:sample_size], y_train[:sample_size])
    end = time() # Get end time
    
    #Calculate the training time
    results['train_time'] = end-start
        
    #Get the predictions on the test set,
    #then get predictions on the first 300 training samples
    start = time() # Get start time
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train[:300])
    end = time() # Get end time
    
    #Calculate the total prediction time
    results['pred_time'] = end-start
            
    #Compute accuracy on the first 300 training samples
    results['acc_train'] = accuracy_score(y_train[:300], predictions_train)
        
    #Compute accuracy on test set
    results['acc_test'] = accuracy_score(y_test, predictions_test)
    
    #Compute AUC score on the first 300 training samples
    results['auc_train'] = roc_auc_score(y_train[:300], predictions_train)
        
    #Compute AUC score on the test set
    results['auc_test'] = roc_auc_score(y_test, predictions_test)
       
    #Success
    #print "{} trained on {} samples.".format(learner.__class__.__name__, sample_size)
        
    # Return the results
    return results

Evaluate classifiers:

In [23]:
#Import supervised learning models from sklearn
import random
import mimic_visuals as vs
from time import time
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split

#Set up results dictionary for classifiers
clf_A = GaussianNB()
clf_B = tree.DecisionTreeClassifier(random_state=42)
clf_C = RandomForestClassifier(random_state =42)
clf_D = AdaBoostClassifier(random_state = 42)
#clf_E = SVC(random_state = 42)

clf_name_list = []
for clf in [clf_A, clf_B, clf_C, clf_D]:
    clf_name = clf.__class__.__name__
    clf_name_list.append(clf_name)







def best_classifier(features, expiry, trials, classifiers):
    #Run through multiple times using random states
    num_trials = trials
    results = {}
    for clf_name in classifiers:
        results[clf_name] = {}
    for trial in range(num_trials):
        state = random.randint(0,100)
        #print "Trial {}: Random State is {}".format(trial+1, state)
    
        #Create test and train samples
        X_train, X_test, y_train, y_test = train_test_split(features, expiry,
                                                                   random_state=random.randint(0,100))
        #Check for correct splitting
        #print "Training set has {} samples.".format(X_train.shape[0])
        #print "Testing set has {} samples.".format(X_test.shape[0])

        #  Initialize the models
        clf_A = GaussianNB()
        clf_B = tree.DecisionTreeClassifier(random_state=state)
        clf_C = RandomForestClassifier(random_state =state)
        clf_D = AdaBoostClassifier(random_state = state)
        #clf_E = SVC(random_state = state)
        
        
    
        samples_100 = int(bench_train.shape[0])

        # Collect results on the learners
        for clf in [clf_A, clf_B, clf_C, clf_D]:
            clf_name = clf.__class__.__name__
            results[clf_name][trial] = \
            train_results(clf, samples_100, X_train, y_train, X_test, y_test)

    # Run metrics visualization for the three supervised learning models chosen
    #vs.evaluate(results, 0.6614, 0.5)

    #Find average AUC scores
    average_scores = {'accuracy': {}, 'auc': {}}
    #print results
    for clf in results:
        auc_scores = 0
        accuracy_scores = 0
        for trial in results[clf]:
            auc_scores+= results[clf][trial]['auc_test']
            accuracy_scores += results[clf][trial]['acc_test']
        average_scores['auc'][clf] = auc_scores/num_trials
        average_scores['accuracy'][clf] = accuracy_scores/num_trials
    

    # Print final AUC scores
    for clf in results:
        print "{} has an average AUC test score of {:.6f}".format(clf, average_scores['auc'][clf])
        print "{} has an average accuracy test score of {:.6f}".format(clf, average_scores['accuracy'][clf])

best_classifier(benchmark_features, expiry, 10, clf_name_list)

RandomForestClassifier has an average AUC test score of 0.688192
RandomForestClassifier has an average accuracy test score of 0.733867
GaussianNB has an average AUC test score of 0.696541
GaussianNB has an average accuracy test score of 0.716981
AdaBoostClassifier has an average AUC test score of 0.690509
AdaBoostClassifier has an average accuracy test score of 0.733910
DecisionTreeClassifier has an average AUC test score of 0.686808
DecisionTreeClassifier has an average accuracy test score of 0.734461


From the benchmark model testing, we find that the GaussianNB has the highest AUC score with 0.6980.  With this as the chosen classification model, we can see whether we can improve this score. 

In [24]:
#Import 'GridSearchCV', 'make_scorer', and any other necessary libraries
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

#Initialize the classifier
clf = tree.DecisionTreeClassifier(random_state=42)

#Split data
X_train, X_test, y_train, y_test = train_test_split(benchmark_features, expiry, random_state=random.randint(0,100))

#Create the parameters list you wish to tune
parameters = {'min_samples_split' : [2,5,8,10], 'max_depth' : [2, 4, 6, 10],
              'min_samples_leaf' : [1,2,3,4]}

#Make an auc_score scoring object
scorer = make_scorer(roc_auc_score)

#Perform grid search on the classifier using 'scorer' as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=scorer)

#Fit the grid search object to the training data and find the optimal parameters
grid_fit = grid_obj.fit(X_train, y_train)

#Get the estimator
best_clf = grid_fit.best_estimator_
print best_clf.get_params()

# Make predictions using the unoptimized and model
predictions = (clf.fit(X_train, y_train)).predict(X_test)
best_predictions = best_clf.predict(X_test)

# Report the before-and-afterscores
print "Unoptimized model\n------"
print "Accuracy score on testing data: {:.4f}".format(accuracy_score(y_test, predictions))
print "AUC score on testing data: {:.4f}".format(roc_auc_score(y_test, predictions))
print "\nOptimized Model\n------"
print "Final accuracy score on the testing data: {:.4f}".format(accuracy_score(y_test, best_predictions))
print "Final AUC score on the testing data: {:.4f}".format(roc_auc_score(y_test, best_predictions))

{'presort': False, 'splitter': 'best', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'criterion': 'gini', 'random_state': 42, 'min_impurity_split': 1e-07, 'max_features': None, 'max_depth': 2, 'class_weight': None}
Unoptimized model
------
Accuracy score on testing data: 0.7343
AUC score on testing data: 0.6855

Optimized Model
------
Final accuracy score on the testing data: 0.7022
Final AUC score on the testing data: 0.6989


### Part 3: Building model including gender and type of insurance
Create models using features that are included in each ICU admission (gender, insurance).  Add these features to age and LOS.

In [25]:
#Select raw features
all_rows_features_raw = dfmimic[['age', 'los', 'gender', 'insurance']]

all_rows_features = pd.get_dummies(all_rows_features_raw)
print list(all_rows_features.columns)

['age_45-64 years', 'age_65-74 years', 'age_75-84 years', 'age_85 years and over', 'age_Under 45 years', 'los_1 day', 'los_10 or more days', 'los_2-3 days', 'los_4-5 days', 'los_6-7 days', 'los_8-9 days', 'gender_F', 'gender_M', 'insurance_Government', 'insurance_Medicaid', 'insurance_Medicare', 'insurance_Private', 'insurance_Self Pay']


In [26]:
#Compare all rows features

best_classifier(all_rows_features, expiry, 10, clf_name_list)

RandomForestClassifier has an average AUC test score of 0.694743
RandomForestClassifier has an average accuracy test score of 0.734487
GaussianNB has an average AUC test score of 0.714523
GaussianNB has an average accuracy test score of 0.705000
AdaBoostClassifier has an average AUC test score of 0.691953
AdaBoostClassifier has an average accuracy test score of 0.734555
DecisionTreeClassifier has an average AUC test score of 0.695021
DecisionTreeClassifier has an average accuracy test score of 0.734702


We can see that adding 'gender' and 'insurance' features has improved the AUC score of the model from 0.6990 to 0.7119.  Let us further look into this to see whether adding one feature or the other changes the AUC score.

In [27]:
#Select raw features
all_rows_gender_raw = dfmimic[['age', 'los', 'gender']]

all_rows_gender = pd.get_dummies(all_rows_gender_raw)
print list(all_rows_gender.columns)

#Find best classifier with 'age' 'los' and 'gender' features
best_classifier(all_rows_gender, expiry, 10, clf_name_list)

['age_45-64 years', 'age_65-74 years', 'age_75-84 years', 'age_85 years and over', 'age_Under 45 years', 'los_1 day', 'los_10 or more days', 'los_2-3 days', 'los_4-5 days', 'los_6-7 days', 'los_8-9 days', 'gender_F', 'gender_M']
RandomForestClassifier has an average AUC test score of 0.684667
RandomForestClassifier has an average accuracy test score of 0.731595
GaussianNB has an average AUC test score of 0.696768
GaussianNB has an average accuracy test score of 0.719640
AdaBoostClassifier has an average AUC test score of 0.686767
AdaBoostClassifier has an average accuracy test score of 0.731827
DecisionTreeClassifier has an average AUC test score of 0.684236
DecisionTreeClassifier has an average accuracy test score of 0.731362


Best AUC score is GaussianNB classifier with AUC score of 0.6964

In [28]:
#Select raw features
all_rows_insurance_raw = dfmimic[['age', 'los', 'insurance']]

all_rows_insurance = pd.get_dummies(all_rows_insurance_raw)
print list(all_rows_insurance.columns)

#Find best classifier with 'age' 'los' and 'gender' features
best_classifier(all_rows_insurance, expiry, 10, clf_name_list)

['age_45-64 years', 'age_65-74 years', 'age_75-84 years', 'age_85 years and over', 'age_Under 45 years', 'los_1 day', 'los_10 or more days', 'los_2-3 days', 'los_4-5 days', 'los_6-7 days', 'los_8-9 days', 'insurance_Government', 'insurance_Medicaid', 'insurance_Medicare', 'insurance_Private', 'insurance_Self Pay']
RandomForestClassifier has an average AUC test score of 0.696328
RandomForestClassifier has an average accuracy test score of 0.735046
GaussianNB has an average AUC test score of 0.712258
GaussianNB has an average accuracy test score of 0.702109
AdaBoostClassifier has an average AUC test score of 0.692204
AdaBoostClassifier has an average accuracy test score of 0.734719
DecisionTreeClassifier has an average AUC test score of 0.696850
DecisionTreeClassifier has an average accuracy test score of 0.735192


Best AUC score is GaussianNB with 0.7128.  This is slightly better than both 'insurance' and 'gender' features added.  

Let us now look at the data with the optional features ('ethnicity', 'marital_status', 'religion', and 'language' added.  For these optional features we will be using the cleaned groups dataframe with rows containing 'NaN', 'UNKOWN', and 'UNABLE TO OBTAIN' values removed.

In [29]:


#Use 'ethnicity' with 'age', 'los', 'gender', and 'insurance'
clean_rows_ethnicity_raw = df_clean_groups[['age', 'gender', 'los', 'insurance', 'ethnicity']]
expiry_clean = df_clean_groups[['expire_flag']]

clean_rows_ethnicity = pd.get_dummies(clean_rows_ethnicity_raw)
print list(clean_rows_ethnicity.columns)

#Find best classifier with 'age' 'los' and 'gender' features
best_classifier(clean_rows_ethnicity, expiry_clean, 10, clf_name_list)

['age_45-64 years', 'age_65-74 years', 'age_75-84 years', 'age_85 years and over', 'age_Under 45 years', 'gender_F', 'gender_M', 'los_1 day', 'los_10 or more days', 'los_2-3 days', 'los_4-5 days', 'los_6-7 days', 'los_8-9 days', 'insurance_Government', 'insurance_Medicaid', 'insurance_Medicare', 'insurance_Private', 'insurance_Self Pay', 'ethnicity_AMERICAN INDIAN/ALASKA NATIVE', 'ethnicity_AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE', 'ethnicity_ASIAN', 'ethnicity_ASIAN - ASIAN INDIAN', 'ethnicity_ASIAN - CAMBODIAN', 'ethnicity_ASIAN - CHINESE', 'ethnicity_ASIAN - FILIPINO', 'ethnicity_ASIAN - JAPANESE', 'ethnicity_ASIAN - KOREAN', 'ethnicity_ASIAN - OTHER', 'ethnicity_ASIAN - VIETNAMESE', 'ethnicity_BLACK/AFRICAN', 'ethnicity_BLACK/AFRICAN AMERICAN', 'ethnicity_BLACK/CAPE VERDEAN', 'ethnicity_BLACK/HAITIAN', 'ethnicity_CARIBBEAN ISLAND', 'ethnicity_HISPANIC OR LATINO', 'ethnicity_HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)', 'ethnicity_HISPANIC/LATINO - COLOMBIAN', 'ethni

G:\ProgramData\Anaconda3\envs\spyder27\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
G:\ProgramData\Anaconda3\envs\spyder27\lib\site-packages\ipykernel\__main__.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForestClassifier has an average AUC test score of 0.595882
RandomForestClassifier has an average accuracy test score of 0.703465
GaussianNB has an average AUC test score of 0.511212
GaussianNB has an average accuracy test score of 0.336264
AdaBoostClassifier has an average AUC test score of 0.601474
AdaBoostClassifier has an average accuracy test score of 0.714027
DecisionTreeClassifier has an average AUC test score of 0.595286
DecisionTreeClassifier has an average accuracy test score of 0.704387


In [30]:
#Use 'marital_status' with 'age', 'los', 'gender', and 'insurance'
clean_rows_marital_status_raw = df_clean_groups[['age', 'gender', 'los', 'insurance', 'marital_status']]
expiry_clean = df_clean_groups[['expire_flag']]

clean_rows_marital_status = pd.get_dummies(clean_rows_marital_status_raw)
print list(clean_rows_marital_status.columns)

#Find best classifier with 'age' 'los' and 'gender' features
best_classifier(clean_rows_marital_status, expiry_clean, 10, clf_name_list)

['age_45-64 years', 'age_65-74 years', 'age_75-84 years', 'age_85 years and over', 'age_Under 45 years', 'gender_F', 'gender_M', 'los_1 day', 'los_10 or more days', 'los_2-3 days', 'los_4-5 days', 'los_6-7 days', 'los_8-9 days', 'insurance_Government', 'insurance_Medicaid', 'insurance_Medicare', 'insurance_Private', 'insurance_Self Pay', 'marital_status_DIVORCED', 'marital_status_LIFE PARTNER', 'marital_status_MARRIED', 'marital_status_SEPARATED', 'marital_status_SINGLE', 'marital_status_WIDOWED']


G:\ProgramData\Anaconda3\envs\spyder27\lib\site-packages\ipykernel\__main__.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForestClassifier has an average AUC test score of 0.593842
RandomForestClassifier has an average accuracy test score of 0.704952
GaussianNB has an average AUC test score of 0.645313
GaussianNB has an average accuracy test score of 0.614818
AdaBoostClassifier has an average AUC test score of 0.602018
AdaBoostClassifier has an average accuracy test score of 0.713820
DecisionTreeClassifier has an average AUC test score of 0.589566
DecisionTreeClassifier has an average accuracy test score of 0.705404


In [31]:
#Use 'language' with 'age', 'los', 'gender', and 'insurance'
clean_rows_language_raw = df_clean_groups[['age', 'gender', 'los', 'insurance', 'language']]
expiry_clean = df_clean_groups[['expire_flag']]

clean_rows_language = pd.get_dummies(clean_rows_language_raw)
print list(clean_rows_language.columns)

#Find best classifier with 'age' 'los' and 'gender' features
best_classifier(clean_rows_language, expiry_clean, 10, clf_name_list)

['age_45-64 years', 'age_65-74 years', 'age_75-84 years', 'age_85 years and over', 'age_Under 45 years', 'gender_F', 'gender_M', 'los_1 day', 'los_10 or more days', 'los_2-3 days', 'los_4-5 days', 'los_6-7 days', 'los_8-9 days', 'insurance_Government', 'insurance_Medicaid', 'insurance_Medicare', 'insurance_Private', 'insurance_Self Pay', 'language_ALBA', 'language_AMER', 'language_ARAB', 'language_CAMB', 'language_CANT', 'language_CAPE', 'language_ENGL', 'language_GREE', 'language_HAIT', 'language_HIND', 'language_ITAL', 'language_KORE', 'language_MAND', 'language_PERS', 'language_POLI', 'language_PORT', 'language_PTUN', 'language_RUSS', 'language_SPAN', 'language_VIET']


G:\ProgramData\Anaconda3\envs\spyder27\lib\site-packages\ipykernel\__main__.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForestClassifier has an average AUC test score of 0.599875
RandomForestClassifier has an average accuracy test score of 0.705103
GaussianNB has an average AUC test score of 0.621243
GaussianNB has an average accuracy test score of 0.668688
AdaBoostClassifier has an average AUC test score of 0.605712
AdaBoostClassifier has an average accuracy test score of 0.713124
DecisionTreeClassifier has an average AUC test score of 0.595268
DecisionTreeClassifier has an average accuracy test score of 0.704726


In [32]:
#Use 'language' with 'age', 'los', 'gender', and 'insurance'
clean_rows_language_raw = df_clean_groups[['age', 'gender', 'los', 'insurance', 'language']]
expiry_clean = df_clean_groups[['expire_flag']]

clean_rows_language = pd.get_dummies(clean_rows_language_raw)
print list(clean_rows_language.columns)

#Find best classifier with 'age' 'los' and 'gender' features
best_classifier(clean_rows_language, expiry_clean, 10, clf_name_list)

['age_45-64 years', 'age_65-74 years', 'age_75-84 years', 'age_85 years and over', 'age_Under 45 years', 'gender_F', 'gender_M', 'los_1 day', 'los_10 or more days', 'los_2-3 days', 'los_4-5 days', 'los_6-7 days', 'los_8-9 days', 'insurance_Government', 'insurance_Medicaid', 'insurance_Medicare', 'insurance_Private', 'insurance_Self Pay', 'language_ALBA', 'language_AMER', 'language_ARAB', 'language_CAMB', 'language_CANT', 'language_CAPE', 'language_ENGL', 'language_GREE', 'language_HAIT', 'language_HIND', 'language_ITAL', 'language_KORE', 'language_MAND', 'language_PERS', 'language_POLI', 'language_PORT', 'language_PTUN', 'language_RUSS', 'language_SPAN', 'language_VIET']


G:\ProgramData\Anaconda3\envs\spyder27\lib\site-packages\ipykernel\__main__.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForestClassifier has an average AUC test score of 0.599606
RandomForestClassifier has an average accuracy test score of 0.706326
GaussianNB has an average AUC test score of 0.618362
GaussianNB has an average accuracy test score of 0.677104
AdaBoostClassifier has an average AUC test score of 0.607302
AdaBoostClassifier has an average accuracy test score of 0.716230
DecisionTreeClassifier has an average AUC test score of 0.594722
DecisionTreeClassifier has an average accuracy test score of 0.705743
